In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
theta_0 = np.array([[np.nan, 1, 1, np.nan],  # s0
                    [np.nan, 1, np.nan, 1],  # s1
                    [np.nan, np.nan, np.nan, 1],  # s2
                    [1, 1, 1, np.nan],  # s3
                    [np.nan, 1, np.nan, 1],  # s4
                    [np.nan, np.nan, 1, 1],  # s5
                    [1, 1, np.nan, np.nan],  # s6
                    [np.nan, np.nan, np.nan, 1],  # s7
                    [np.nan, np.nan, np.nan, np.nan], # s8
                    ])

In [ ]:
def softmax_convert_into_pi_from_theta(theta):
    '''비율 계산에 소프트맥스 함수 사용'''

    beta = 1.0
    [m, n] = theta.shape  # theta의 행렬 크기를 구함
    pi = np.zeros((m, n))

    exp_theta = np.exp(beta * theta)  # theta를 exp(theta)로 변환

    for i in range(0, m):
        # pi[i, :] = theta[i, :] / np.nansum(theta[i, :])
        # 단순 비율을 계산하는 코드

        pi[i, :] = exp_theta[i, :] / np.nansum(exp_theta[i, :])
        # softmax로 계산하는 코드

    pi = np.nan_to_num(pi)  # nan을 0으로 변환

    return pi

direction = ["up", "right", "down", "left"]

def get_action_and_next_s(pi, s):  # 폭
    n = int(np.sqrt(pi.shape[0]))
    
    # pi[s,:]의 확률을 따라, direction값이 선택된다
    next_direction = np.random.choice(direction, p=pi[s, :])

    if next_direction == "up":
        action = 0
        s_next = s - n  # 위로 이동하면 상태값이 3 줄어든다
    elif next_direction == "right":
        action = 1
        s_next = s + 1  # 오른쪽으로 이동하면 상태값이 1 늘어난다
    elif next_direction == "down":
        action = 2
        s_next = s + n  # 아래로 이동하면 상태값이 3 늘어난다
    elif next_direction == "left":
        action = 3
        s_next = s - 1  # 왼쪽으로 이동하면 상태값이 1 줄어든다

    return [action, s_next]


def goal_maze_ret_s_a(pi):
    goal = np.argmin(np.nansum(theta_0, axis=1)) # 8
    s = 0  # 시작 지점
    s_a_history = []

    while (1):  # 목표 지점에 이를 때까지 반복
        [action, next_s] = get_action_and_next_s(pi, s)

        s_a_history.append([s, action, direction[action], next_s])
        
        if next_s == goal:  # 목표 지점에 이르면 종료
            s_a_history.append([goal, np.nan])
            break
        else:
            s = next_s

    return s_a_history


def update_theta(theta, pi, s_a_history):
    eta = 0.1 # 학습률
    T = len(s_a_history) - 1  # 목표 지점에 이르기까지 걸린 단계 수

    [m, n] = theta.shape  # theta의 행렬 크기를 구함
    delta_theta = theta.copy()  # Δtheta를 구할 준비, 포인터 참조이므로 delta_theta = theta로는 안됨

    # delta_theta를 요소 단위로 계산
    for i in range(0, m):
        for j in range(0, n):
            if not(np.isnan(theta[i, j])):  # theta가 nan이 아닌 경우

                SA_i = [SA for SA in s_a_history if SA[0] == i]
                # 히스토리에서 상태 i인 것만 모아오는 리스트 컴프리헨션

                SA_ij = [SA for SA in s_a_history if SA[0:2] == [i, j]]
                # 상태 i에서 행동 j를 취한 경우만 모음

                N_i = len(SA_i)  # 상태 i에서 모든 행동을 취한 횟수
                N_ij = len(SA_ij)  # 상태 i에서 행동 j를 취한 횟수 
                
                delta_theta[i, j] = (N_ij - pi[i, j] * N_i) / T
                #delta_theta[i, j] = N_ij / T

    new_theta = theta + eta * delta_theta

    return new_theta

In [ ]:
softmax_convert_into_pi_from_theta(theta_0)

array([[0.        , 0.5       , 0.5       , 0.        ],
       [0.        , 0.5       , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.33333333, 0.33333333, 0.33333333, 0.        ],
       [0.        , 0.5       , 0.        , 0.5       ],
       [0.        , 0.        , 0.5       , 0.5       ],
       [0.5       , 0.5       , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [ ]:
pi_0 = softmax_convert_into_pi_from_theta(theta_0)
s_a_history = goal_maze_ret_s_a(pi_0)
print(s_a_history)
print("목표 지점에 이르기까지 걸린 단계 수는 " + str(len(s_a_history) - 1) + "단계입니다")

[[0, 1, 'right', 1], [1, 1, 'right', 2], [2, 3, 'left', 1], [1, 3, 'left', 0], [0, 2, 'down', 3], [3, 2, 'down', 6], [6, 1, 'right', 7], [7, 3, 'left', 6], [6, 1, 'right', 7], [7, 3, 'left', 6], [6, 1, 'right', 7], [7, 3, 'left', 6], [6, 1, 'right', 7], [7, 3, 'left', 6], [6, 0, 'up', 3], [3, 0, 'up', 0], [0, 1, 'right', 1], [1, 3, 'left', 0], [0, 1, 'right', 1], [1, 3, 'left', 0], [0, 1, 'right', 1], [1, 1, 'right', 2], [2, 3, 'left', 1], [1, 1, 'right', 2], [2, 3, 'left', 1], [1, 1, 'right', 2], [2, 3, 'left', 1], [1, 3, 'left', 0], [0, 1, 'right', 1], [1, 3, 'left', 0], [0, 2, 'down', 3], [3, 0, 'up', 0], [0, 2, 'down', 3], [3, 1, 'right', 4], [4, 3, 'left', 3], [3, 2, 'down', 6], [6, 1, 'right', 7], [7, 3, 'left', 6], [6, 0, 'up', 3], [3, 0, 'up', 0], [0, 1, 'right', 1], [1, 3, 'left', 0], [0, 2, 'down', 3], [3, 2, 'down', 6], [6, 0, 'up', 3], [3, 2, 'down', 6], [6, 0, 'up', 3], [3, 2, 'down', 6], [6, 0, 'up', 3], [3, 1, 'right', 4], [4, 3, 'left', 3], [3, 2, 'down', 6], [6, 0, 'up

In [ ]:
# 정책 수정 한번 더  수정
pi = softmax_convert_into_pi_from_theta(theta_0)
print(pi)
s_a_history = goal_maze_ret_s_a(pi) 
theta = update_theta(theta_0, pi, s_a_history)
pi = softmax_convert_into_pi_from_theta(theta)
print(pi)

[[0.         0.5        0.5        0.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.         0.5        0.5       ]
 [0.5        0.5        0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         0.        ]]
[[0.         0.4983871  0.5016129  0.        ]
 [0.         0.50282255 0.         0.49717745]
 [0.         0.         0.         1.        ]
 [0.33351192 0.33243781 0.33405027 0.        ]
 [0.         0.50040323 0.         0.49959677]
 [0.         0.         0.5        0.5       ]
 [0.4983871  0.5016129  0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         0.        ]]


In [ ]:
# 4000번 수행
pi = softmax_convert_into_pi_from_theta(theta_0)
theta = theta_0.copy()
for i in range(4000) :
  s_a_history = goal_maze_ret_s_a(pi) 
  theta = update_theta(theta, pi, s_a_history)
  pi = softmax_convert_into_pi_from_theta(theta)   
  print("목표 지점에 이르기까지 걸린 단계 수는 " + str(len(s_a_history) - 1) + "단계입니다")
  

목표 지점에 이르기까지 걸린 단계 수는 38단계입니다
목표 지점에 이르기까지 걸린 단계 수는 70단계입니다
목표 지점에 이르기까지 걸린 단계 수는 28단계입니다
목표 지점에 이르기까지 걸린 단계 수는 70단계입니다
목표 지점에 이르기까지 걸린 단계 수는 6단계입니다
목표 지점에 이르기까지 걸린 단계 수는 52단계입니다
목표 지점에 이르기까지 걸린 단계 수는 12단계입니다
목표 지점에 이르기까지 걸린 단계 수는 26단계입니다
목표 지점에 이르기까지 걸린 단계 수는 108단계입니다
목표 지점에 이르기까지 걸린 단계 수는 22단계입니다
목표 지점에 이르기까지 걸린 단계 수는 20단계입니다
목표 지점에 이르기까지 걸린 단계 수는 20단계입니다
목표 지점에 이르기까지 걸린 단계 수는 8단계입니다
목표 지점에 이르기까지 걸린 단계 수는 14단계입니다
목표 지점에 이르기까지 걸린 단계 수는 50단계입니다
목표 지점에 이르기까지 걸린 단계 수는 60단계입니다
목표 지점에 이르기까지 걸린 단계 수는 112단계입니다
목표 지점에 이르기까지 걸린 단계 수는 66단계입니다
목표 지점에 이르기까지 걸린 단계 수는 44단계입니다
목표 지점에 이르기까지 걸린 단계 수는 8단계입니다
목표 지점에 이르기까지 걸린 단계 수는 10단계입니다
목표 지점에 이르기까지 걸린 단계 수는 14단계입니다
목표 지점에 이르기까지 걸린 단계 수는 50단계입니다
목표 지점에 이르기까지 걸린 단계 수는 14단계입니다
목표 지점에 이르기까지 걸린 단계 수는 18단계입니다
목표 지점에 이르기까지 걸린 단계 수는 14단계입니다
목표 지점에 이르기까지 걸린 단계 수는 16단계입니다
목표 지점에 이르기까지 걸린 단계 수는 44단계입니다
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
목표 지점에 이르기까지 걸린 단계 수는 30단계입니다
목표 지점에 이르기까지 걸린 단계 수는 16단계입니다
목표 지점에 이르기까지 걸린 단계 수는 38단계입니다
목표 지점에 이르기까지 걸린 단계 수는 18단계입니다
목표 지점에 이르기까지

In [ ]:
print(pi)

[[0.         0.01375567 0.98624433 0.        ]
 [0.         0.33190269 0.         0.66809731]
 [0.         0.         0.         1.        ]
 [0.01246745 0.980297   0.00723555 0.        ]
 [0.         0.98190097 0.         0.01809903]
 [0.         0.         0.98187554 0.01812446]
 [0.63508962 0.36491038 0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         0.        ]]


In [ ]:
s_a_history = goal_maze_ret_s_a(pi)
print(s_a_history)
print("목표 지점에 이르기까지 걸린 단계 수는 " + str(len(s_a_history) - 1) + "단계입니다")

[[0, 2, 'down', 3], [3, 1, 'right', 4], [4, 1, 'right', 5], [5, 2, 'down', 8], [8, nan]]
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다


In [ ]:
#                      상,  우, 하,  좌                  
theta_0 = np.array([[np.nan, 1, 1, np.nan],  # s0
                    [np.nan, 1, np.nan, 1],  # s1
                    [np.nan, np.nan, np.nan, 1],  # s2
                    [1, 1, 1, np.nan],  # s3
                    [np.nan, 1, np.nan, 1],  # s4
                    [np.nan, np.nan, 1, 1],  # s5
                    [1, 1, np.nan, np.nan],  # s6
                    [np.nan, np.nan, np.nan, 1],  # s7
                    [np.nan, np.nan, np.nan, np.nan], # s8
                    ])

In [ ]:
def simple_convert_into_pi_from_theta(theta):
    
    [m, n] = theta.shape 
    pi = np.zeros((m, n))
    for i in range(0, m):
        pi[i, :] = theta[i, :] / np.nansum(theta[i, :]) 

    pi = np.nan_to_num(pi) 

    return pi
pi_0 = simple_convert_into_pi_from_theta(theta_0)    
print(pi_0)

[[0.         0.5        0.5        0.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333333 0.33333333 0.33333333 0.        ]
 [0.         0.5        0.         0.5       ]
 [0.         0.         0.5        0.5       ]
 [0.5        0.5        0.         0.        ]
 [0.         0.         0.         1.        ]
 [0.         0.         0.         0.        ]]


In [ ]:
[a, b] = theta_0.shape
Q = np.random.rand(a, b) * theta_0 * 0.1
print(Q)

[[       nan 0.01204595 0.02998585        nan]
 [       nan 0.06904817        nan 0.03612914]
 [       nan        nan        nan 0.09941994]
 [0.03742723 0.02237555 0.07126915        nan]
 [       nan 0.07849272        nan 0.04858171]
 [       nan        nan 0.0471391  0.04453754]
 [0.03309248 0.04316363        nan        nan]
 [       nan        nan        nan 0.00321384]
 [       nan        nan        nan        nan]]


In [ ]:
# ε-greedy 알고리즘 구현

def get_action(s, Q, epsilon, pi_0):
    direction = ["up", "right", "down", "left"]

    if np.random.rand() < epsilon:
        next_direction = np.random.choice(direction, p=pi_0[s, :])
    else:
        next_direction = direction[np.nanargmax(Q[s, :])]

    if next_direction == "up":
        action = 0
    elif next_direction == "right":
        action = 1
    elif next_direction == "down":
        action = 2
    elif next_direction == "left":
        action = 3

    return action


def get_s_next(s, a):
    direction = ["up", "right", "down", "left"]
    next_direction = direction[a] 
    
    if next_direction == "up":
        s_next = s - 3
    elif next_direction == "right":
        s_next = s + 1
    elif next_direction == "down":
        s_next = s + 3
    elif next_direction == "left":
        s_next = s - 1

    return s_next

In [ ]:
# Q러닝 알고리즘으로 행동가치 함수 Q를 수정

def Q_learning(s, a, r, s_next, Q, eta, gamma):

    if s_next == 8:  # 목표 지점에 도달한 경우
        Q[s, a] = Q[s, a] + eta * (r - Q[s, a])

    else:
        Q[s, a] = Q[s, a] + eta * (r + gamma * np.nanmax(Q[s_next,: ]) - Q[s, a])

    return Q

In [ ]:
def goal_maze_ret_s_a_Q(Q, epsilon, eta, gamma, pi):
    direction = ["up", "right", "down", "left"]
    s = 0 
    a = a_next = get_action(s, Q, epsilon, pi) 

    s_a_history = []

    while (1): 
        a = a_next
        s_next = get_s_next(s, a)
        s_a_history.append([s, a,  direction[a], s_next])

        if s_next == 8:
            r = 1
            a_next = np.nan
        else:
            r = 0
            a_next = get_action(s_next, Q, epsilon, pi)          

        Q = Q_learning(s, a, r, s_next, Q, eta, gamma)
        
        if s_next == 8:   break
        else:  s = s_next

    return [s_a_history, Q]

In [ ]:
pi_0 = simple_convert_into_pi_from_theta(theta_0)    
[a, b] = theta_0.shape
Q = np.random.rand(a, b) * theta_0 * 0.1


eta = 0.1  # 학습률
gamma = 0.9  # 시간할인율
epsilon = 0.5  # ε-greedy 알고리즘 epsilon 초깃값

for episode in range(20) :
    print("에피소드: " + str(episode+1))
    epsilon = epsilon / 2

    [s_a_history, Q] = goal_maze_ret_s_a_Q(Q, epsilon, eta, gamma, pi_0)

    print("목표 지점에 이르기까지 걸린 단계 수는 " + str(len(s_a_history)) + "단계입니다")
    
print(s_a_history)

에피소드: 1
목표 지점에 이르기까지 걸린 단계 수는 52단계입니다
에피소드: 2
목표 지점에 이르기까지 걸린 단계 수는 38단계입니다
에피소드: 3
목표 지점에 이르기까지 걸린 단계 수는 120단계입니다
에피소드: 4
목표 지점에 이르기까지 걸린 단계 수는 8단계입니다
에피소드: 5
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 6
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 7
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 8
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 9
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 10
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 11
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 12
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 13
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 14
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 15
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 16
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 17
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 18
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 19
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
에피소드: 20
목표 지점에 이르기까지 걸린 단계 수는 4단계입니다
[[0, 2, 'down', 3], [3, 1, 'right', 4], [4, 1, 'right', 5], [5, 2, 'down', 8]]
